In [67]:
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
import re
import string
from nltk.tokenize import TweetTokenizer

In [68]:
df_vis = pd.read_csv('vis_fin.csv')
df_exb = pd.read_csv('exhib_fin.csv')

In [69]:
pd.set_option('display.max_columns', None)
df_exb.head()

,id,company_repName,company_name,blood_group,mobile_no,email,designation,country,password,exhibitor,role,pro_category,createdAt,updatedAt,company_profile,company_logo,is_confirmed,profile_completion,city,state,address_line1,address_line2,address_line3,fascia,website,post_code,gst,host_qty,hostess_qty,single_phase_connection,three_phase_connection,indemnity_undertaking,fever,cough,breathlessness,sore_throat_running_nose,body_ache,other_symptom,health_undertaking_accepted,exhibitor_deliverables_correct,vaccine_certificate,stall_design,stall_design_correct,alternative_email,title,first_name,last_name,oem_user_profile,oem_fascia,oem_exhibitor_badges,oem_power_requirement,oem_furniture_requirement,oem_host_hostess,oem_bare_space_stall_design,oem_booth_contractor,oem_visa,oem_indemnity_undertaking,oem_health_undertaking,oem_participation,oem_catalog_entry,company_repProfile,option_a,option_b,host,currentForm,currentFormIndex,booth_contractor_applicable,visa_applicable,firebaseToken,socket_id,catalogue_mobile_no,catalogue_email,contact_person,contact_person_designation,furniture_applicable,account_deletion_request
0,21,SHANTANU KHANNA,"ISO 9001:2015 , ISI, IBR, MSME, NSIC",B+,+919041757000,info@valvcointernational.com,Director Technical,India,$2a$10$SamJoY2Fa/Ss78XYMKA3Du8ZXdZwQpnUrg95eGL...,exhibitor,exhibitor,Ancillaries,2021-12-22 08:46:10,2021-12-22 08:53:21,"Greetings Sir,\n\n\n\nFounded in 1995, VALVCO ...",https://api.boilerworldexpo.com/images/red log...,0,0,Jalandhar,Punjab,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,USER PROFILE,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,37,Deshmane Prakash Murlidhar,VRSl,B+,+919503007843,pmdeshmane1983@gmail.com,Dy.Chief Engineer,India,$2a$10$aMqJo7UoM2G3Ns.B/HXvyOcfp.y3v1IMYxHEwCq...,exhibitor,exhibitor,Technology&serviceprovider,2021-12-23 21:55:20,2022-01-19 10:24:56,VRSL is one largest sugar and power industry.,NaN,0,0,Karmala,Maharastra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,USER PROFILE,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,40,Shivender Singh Chauhan,Moglix,AB+,+918585010200,shivenchauhan1990@gmail.com,Senior Manager,India,$2a$10$PGrp1i9gm6VyzhrUEdwsc.kUm8JgtHJtjF10bnP...,exhibitor,exhibitor,Boilermanufacturer,2021-12-27 06:08:16,2021-12-27 06:10:11,Moglix Profile:\nMoglix is India's largest and...,https://api.boilerworldexpo.com/images/Moglix ...,0,0,Gurgaon,Haryana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,USER PROFILE,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,42,Karan Medappa,Vishwa Samudra Shipment Serv. Pvt. Ltd.,B+,+919886508498,karan.medappa@vishwasamudra.in,VP: Head - Communications,India,$2a$10$zMRY3KV3UQYJ.uauHdmfm.sfmg7ALLAUR8bPdCi...,exhibitor,exhibitor,Ancillaries,2021-12-28 09:27:49,2021-12-28 09:31:39,"The Vishwa Samudra Group, an Indian conglomera...",https://api.boilerworldexpo.com/images/FuelSpe...,0,0,Hyderabad,Telangana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,USER PROFILE,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,45,Manohar C,UE Systems IMENA Pvt ltd,A+,07995019284,admin.imena@uesystems.com,Director,India,$2a$10$64YjGKex.0vQdVrHDUQgt.4yV/889j12WU.sK.x...,exhibitor,exhibitor,Technology&serviceprovider,2022-01-03 13:16:50,2022-09-05 09:49:15,The Most Trusted Source in Ultrasonic Plant Re...,https://api.boilerworldexpo.com/images/UE Syst...,1,0,Hyderabad,Telangana,"4th Floor, Saikrupa Arcade, Kavuri Hills",Madhapur,NaN,UE Systems IMENA Private limited,www.uesystems.com,500033,36AABCU8120H1ZF,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,1,1,NaN,NaN,0,aalekhb@uesystems.com,Mr,Manohar,C,1,1,1,0,0,1,0,1,1,1,1,0,1,NaN,0,1,"""[{\""category\"":\""Basic Host\"",\""rate_per_day\...",INDEMNITY/UNDERTAKING,10,1,0,NaN,NaN,9491073736,manoharc@uesystems.com,Manohar Chidurala,Director

In [70]:
df_exb = df_exb.head(500)

In [71]:
df_vis.head()


,Unnamed: 0,id,company_repName,company_name,blood_group,mobile_no,email,designation,country,password,profession,role,createdAt,updatedAt,city,state,vaccine_certificate,id_certificate,company_repProfile,firebaseToken,socket_id,about_me,account_deletion_request,pastinter
0,0,3,Abhijit Alok,Kriti Trade Impex Pvt Ltd New Delhi,B+,9.195824e+11,abhijit.a@kimpexflow.com,Assistant Manager,India,$2a$10$6bVOOQKdY2/Bo1Vu0pp7Qez1qirp81GMFfwHvns...,Engineer/Operatorlookingfornewtechnology,visitor,2021-12-14 10:54:33,2021-12-14 10:54:33,Delhi,Delhi,NaN,NaN,NaN,NaN,NaN,".\n\nDriven by innovation, I leverage technolo...",0,"['NEVCO ENGINEERS PVT LTD', 'AVCON CONTROLS PV..."
1,1,4,SANJAY TORE,MTAR Technologies Ltd,O+,9.191775e+11,sanjay.tore@mtar.in,Business Development Manager,India,$2a$10$4K2KOgHWVCWxuWwoUB0hZuuAOL1RuX/oD1rSppN...,Purchase personnel from boiler manufacturing c...,visitor,2021-12-14 11:40:34,2022-08-22 09:30:56,Hyderabad,Telanagana,NaN,NaN,NaN,NaN,NaN,.\n\nDriven procurement professional with 5+ y...,0,"['Tefugen Technologies Pvt Ltd', 'Star-Mech Co..."
2,2,5,Sanjay Dnyanu Pawar,Utopian sugars Ltd,A+,9.183291e+11,sdpawar22@gmail.com,Boiler Attendant,India,$2a$10$Kcix6Sn0B2OT3sBdSiFOP.i5sbK3jKB/w/FhVqO...,Engineer/Operatorlookingfornewtechnology,visitor,2021-12-15 07:24:23,2021-12-15 07:24:23,Solapur,Maharashtra,NaN,NaN,NaN,NaN,NaN,".\n\nDriven by innovation, I leverage technolo...",0,"['Dugar Tubes Pvt Ltd', 'SKM IMPEX (A Division..."
3,3,6,Kishor Kumbhar,Berkeley Energy Commercial and Industrial Soli...,B+,9.180568e+11,kishor.kumbhar@be-cis.com,Assistant Manager - BD,India,$2a$10$Sr0tGU/i4T7ApyGz0Iy88Odjy.Gh8NPX3OFE.9n...,RenewableEnergyProjectDeveloper,visitor,2021-12-20 09:38:00,2022-08-30 09:39:01,Pune,Maharashtra,NaN,NaN,NaN,NaN,NaN,.\n\nKishor Kumbhar is a passionate Renewable ...,0,"['Nirmal Energy Limited / BALTUR', 'RIBO INDUS..."
4,4,7,Hardik Ambaliya,Zeel Engineering and Fabrication,A+,9.197272e+11,zeel.fabrication@gmail.com,Founder,India,$2a$10$gmA1xn96l44vvfmA2Qbeke0dFC3JUPAQ2etjN60...,PurchasePersonnelFromBoilerManufacturingCompany,visitor,2021-12-20 17:46:55,2021-12-20 17:46:55,Surat,Gujarat,NaN,NaN,NaN,NaN,NaN,".\n\nDriven by innovation, I leverage technolo...",0,"['LLOYDS STEELS INDUSTRIES LTD', 'Manas Micros..."


In [72]:
df_vis.info

<bound method DataFrame.info of        Unnamed: 0     id         company_repName  \
0               0      3            Abhijit Alok   
1               1      4             SANJAY TORE   
2               2      5    Sanjay Dnyanu Pawar    
3               3      6         Kishor Kumbhar    
4               4      7         Hardik Ambaliya   
...           ...    ...                     ...   
11660       12160  12242              8894308351   
11661       12161  12243             pintu yadav   
11662       12162  12244      deepak suryawanshi   
11663       12163  12245  NIKUNJ SINGHAL Singhal   
11664       12164  12246            devang patel   

                                            company_name blood_group  \
0                    Kriti Trade Impex Pvt Ltd New Delhi          B+   
1                                 MTAR Technologies Ltd           O+   
2                                    Utopian sugars Ltd           A+   
3      Berkeley Energy Commercial and Industrial Soli..

In [73]:
df_vis['text'] = df_vis['role']+df_vis['about_me']+df_vis['pastinter']

In [74]:
df_vis.dropna(subset=["about_me"], inplace=True)


In [75]:
df_vis.head()

,Unnamed: 0,id,company_repName,company_name,blood_group,mobile_no,email,designation,country,password,profession,role,createdAt,updatedAt,city,state,vaccine_certificate,id_certificate,company_repProfile,firebaseToken,socket_id,about_me,account_deletion_request,pastinter,text
0,0,3,Abhijit Alok,Kriti Trade Impex Pvt Ltd New Delhi,B+,9.195824e+11,abhijit.a@kimpexflow.com,Assistant Manager,India,$2a$10$6bVOOQKdY2/Bo1Vu0pp7Qez1qirp81GMFfwHvns...,Engineer/Operatorlookingfornewtechnology,visitor,2021-12-14 10:54:33,2021-12-14 10:54:33,Delhi,Delhi,NaN,NaN,NaN,NaN,NaN,".\n\nDriven by innovation, I leverage technolo...",0,"['NEVCO ENGINEERS PVT LTD', 'AVCON CONTROLS PV...","visitor.\n\nDriven by innovation, I leverage t..."
1,1,4,SANJAY TORE,MTAR Technologies Ltd,O+,9.191775e+11,sanjay.tore@mtar.in,Business Development Manager,India,$2a$10$4K2KOgHWVCWxuWwoUB0hZuuAOL1RuX/oD1rSppN...,Purchase personnel from boiler manufacturing c...,visitor,2021-12-14 11:40:34,2022-08-22 09:30:56,Hyderabad,Telanagana,NaN,NaN,NaN,NaN,NaN,.\n\nDriven procurement professional with 5+ y...,0,"['Tefugen Technologies Pvt Ltd', 'Star-Mech Co...",visitor.\n\nDriven procurement professional wi...
2,2,5,Sanjay Dnyanu Pawar,Utopian sugars Ltd,A+,9.183291e+11,sdpawar22@gmail.com,Boiler Attendant,India,$2a$10$Kcix6Sn0B2OT3sBdSiFOP.i5sbK3jKB/w/FhVqO...,Engineer/Operatorlookingfornewtechnology,visitor,2021-12-15 07:24:23,2021-12-15 07:24:23,Solapur,Maharashtra,NaN,NaN,NaN,NaN,NaN,".\n\nDriven by innovation, I leverage technolo...",0,"['Dugar Tubes Pvt Ltd', 'SKM IMPEX (A Division...","visitor.\n\nDriven by innovation, I leverage t..."
3,3,6,Kishor Kumbhar,Berkeley Energy Commercial and Industrial Soli...,B+,9.180568e+11,kishor.kumbhar@be-cis.com,Assistant Manager - BD,India,$2a$10$Sr0tGU/i4T7ApyGz0Iy88Odjy.Gh8NPX3OFE.9n...,RenewableEnergyProjectDeveloper,visitor,2021-12-20 09:38:00,2022-08-30 09:39:01,Pune,Maharashtra,NaN,NaN,NaN,NaN,NaN,.\n\nKishor Kumbhar is a passionate Renewable ...,0,"['Nirmal Energy Limited / BALTUR', 'RIBO INDUS...",visitor.\n\nKishor Kumbhar is a passionate Ren...
4,4,7,Hardik Ambaliya,Zeel Engineering and Fabrication,A+,9.197272e+11,zeel.fabrication@gmail.com,Founder,India,$2a$10$gmA1xn96l44vvfmA2Qbeke0dFC3JUPAQ2etjN60...,PurchasePersonnelFromBoilerManufacturingCompany,visitor,2021-12-20 17:46:55,2021-12-20 17:46:55,Surat,Gujarat,NaN,NaN,NaN,NaN,NaN,".\n\nDriven by innovation, I leverage technolo...",0,"['LLOYDS STEELS INDUSTRIES LTD', 'Manas Micros...","visitor.\n\nDriven by innovation, I leverage t..."


In [76]:

def process_text(text):
    """Process text function.
    Input:
        text: a string containing the text
    Output:
        processed_text: a list of words containing the processed text

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')

    # remove stock market tickers like $GE
    text = re.sub(r'\$\w*', '', text)
    # remove old style retweet text "RT"
    text = re.sub(r'^RT[\s]+', '', text)
    # remove hyperlinks    
    text = re.sub(r'https?://[^\s\n\r]+', '', text)
    # remove hashtags
    # only removing the hash # sign from the word
    text = re.sub(r'#', '', text)
    # tokenize text
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    text_tokens = tokenizer.tokenize(text)

    processed_text = ""
    for word in text_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # processed_text.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            processed_text+= " " + stem_word
    
    return processed_text

nltk.download('stopwords')  

[nltk_data] Downloading package stopwords to C:\Users\Abhyudya
[nltk_data]     Bhatnagar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [77]:
df_exb['text'] = None
for idx, row in df_exb.iterrows():
    df_exb.at[idx, 'text'] = row['pro_category'] + ' ' + row['company_profile']  

In [78]:
print(process_text(df_exb['text'][0]))

 ancillari greet sir found 1995 valvco intern set prime manufactur high-grad ibr certifi isi mark industri commerci valv iso 9001:2015 certifi co forese huge gap market qualiti valv competit price valvco work hard give custom valu money long-term depend product name ‘ qinn valv ’ critic oper inde achiev overwhelm victori offer boiler mount gate globe valv butterfli valv cock petroleum line valv fluid line fit · bronz gun metal · cast iron · cast steel · forg steel · stainless steel · forg brass “ qinn ” brand product sold full guarante manufactur defect approv vendor · rail coach factori kapurthala · militari engin servic southern western command ·


In [79]:
df_exb["preprocessed_text"] = df_exb["text"].apply(process_text)


In [80]:
df_exb.head(5)

,id,company_repName,company_name,blood_group,mobile_no,email,designation,country,password,exhibitor,role,pro_category,createdAt,updatedAt,company_profile,company_logo,is_confirmed,profile_completion,city,state,address_line1,address_line2,address_line3,fascia,website,post_code,gst,host_qty,hostess_qty,single_phase_connection,three_phase_connection,indemnity_undertaking,fever,cough,breathlessness,sore_throat_running_nose,body_ache,other_symptom,health_undertaking_accepted,exhibitor_deliverables_correct,vaccine_certificate,stall_design,stall_design_correct,alternative_email,title,first_name,last_name,oem_user_profile,oem_fascia,oem_exhibitor_badges,oem_power_requirement,oem_furniture_requirement,oem_host_hostess,oem_bare_space_stall_design,oem_booth_contractor,oem_visa,oem_indemnity_undertaking,oem_health_undertaking,oem_participation,oem_catalog_entry,company_repProfile,option_a,option_b,host,currentForm,currentFormIndex,booth_contractor_applicable,visa_applicable,firebaseToken,socket_id,catalogue_mobile_no,catalogue_email,contact_person,contact_person_designation,furniture_applicable,account_deletion_request,text,preprocessed_text
0,21,SHANTANU KHANNA,"ISO 9001:2015 , ISI, IBR, MSME, NSIC",B+,+919041757000,info@valvcointernational.com,Director Technical,India,$2a$10$SamJoY2Fa/Ss78XYMKA3Du8ZXdZwQpnUrg95eGL...,exhibitor,exhibitor,Ancillaries,2021-12-22 08:46:10,2021-12-22 08:53:21,"Greetings Sir,\n\n\n\nFounded in 1995, VALVCO ...",https://api.boilerworldexpo.com/images/red log...,0,0,Jalandhar,Punjab,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,USER PROFILE,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,"Ancillaries Greetings Sir,\n\n\n\nFounded in 1...",ancillari greet sir found 1995 valvco intern ...
1,37,Deshmane Prakash Murlidhar,VRSl,B+,+919503007843,pmdeshmane1983@gmail.com,Dy.Chief Engineer,India,$2a$10$aMqJo7UoM2G3Ns.B/HXvyOcfp.y3v1IMYxHEwCq...,exhibitor,exhibitor,Technology&serviceprovider,2021-12-23 21:55:20,2022-01-19 10:24:56,VRSL is one largest sugar and power industry.,NaN,0,0,Karmala,Maharastra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,USER PROFILE,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,Technology&serviceprovider VRSL is one largest...,technolog serviceprovid vrsl one largest suga...
2,40,Shivender Singh Chauhan,Moglix,AB+,+918585010200,shivenchauhan1990@gmail.com,Senior Manager,India,$2a$10$PGrp1i9gm6VyzhrUEdwsc.kUm8JgtHJtjF10bnP...,exhibitor,exhibitor,Boilermanufacturer,2021-12-27 06:08:16,2021-12-27 06:10:11,Moglix Profile:\nMoglix is India's largest and...,https://api.boilerworldexpo.com/images/Moglix ...,0,0,Gurgaon,Haryana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,USER PROFILE,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,Boilermanufacturer Moglix Profile:\nMoglix is ...,boilermanufactur moglix profil moglix india' ...
3,42,Karan Medappa,Vishwa Samudra Shipment Serv. Pvt. Ltd.,B+,+919886508498,karan.medappa@vishwasamudra.in,VP: Head - Communications,India,$2a$10$zMRY3KV3UQYJ.uauHdmfm.sfmg7ALLAUR8bPdCi...,exhibitor,exhibitor,Ancillaries,2021-12-28 09:27:49,2021-12-28 09:31:39,"The Vishwa Samudra Group, an Indian conglomera...",https://api.boilerworldexpo.com/images/FuelSpe...,0,0,Hyderabad,Telangana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,USER PROFILE,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,"Ancillaries The Vishwa Samudra Group, an India...",ancillari vishwa samudra group indian conglom...
4,45,Manohar C,UE Systems IMENA Pvt ltd,A+,07995019284,admin.imena@uesystems.com,Director,India,$2a$10$64YjGKex.0vQdVrHDUQgt.4yV/889j12WU.sK.x...,exhibitor,exhibitor,Technology&serviceprovider,2022-01-03 13:16:50,2022-09-05 09:49:15,The Most Trusted Source in Ultrasonic Plant Re...,https://api.boilerworldexpo.com/images/UE Syst...

In [60]:
sentences = [text.split() for text in df_exb['preprocessed_text']]
model = Word2Vec(sentences, min_count=1)

# Vectorize the processed text
vectorized_text = []
for text in sentences:
    vector = []
    for word in text:
        if word in model.wv:
            vector.append(model.wv[word])
    vectorized_text.append(vector)

# Assign the vectorized text back to the dataframe
df_exb['vectorized_text'] = vectorized_text

In [81]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_exb['preprocessed_text'])

# Convert sparse matrix to dense numpy array
df_exb['vectorized_text'] = X.toarray().tolist()


In [82]:
df_vis["preprocessed_text"] = df_vis["text"].apply(process_text)


In [83]:
from gensim.models import Word2Vec

In [63]:
# Train word2vec model
sentences = [text.split() for text in df_vis['preprocessed_text']]
model = Word2Vec(sentences, min_count=1)

# Vectorize the processed text
vectorized_text = []
for text in sentences:
    vector = []
    for word in text:
        if word in model.wv:
            vector.append(model.wv[word])
    vectorized_text.append(vector)

# Assign the vectorized text back to the dataframe
df_vis['vectorized_text'] = vectorized_text

In [89]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(n_neighbors=10,metric="cosine")
model.fit(X)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [64]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend(text):
    for idx, row in df_exb.iterrows():
        cosine_similarity_score = cosine_similarity(text, [row['vectorized_text']])[0][0]
        df_exb.at[idx, 'cosine_similarity_score'] = cosine_similarity_score
    return df_exb.sort_values(by='cosine_similarity_score', ascending=False).head(10)

In [90]:

def find_related_articles(keywords):
    # Preprocess the user input keywords
    processed_keywords = process_text(keywords)
    
    # Transform the user input keywords into TF-IDF features
    keyword_features = vectorizer.transform([processed_keywords])
    
    # Find the nearest neighbors to the user input keywords
    distances, indices = model.kneighbors(keyword_features)
    
    # Get the related articles based on the nearest neighbors
    related_articles = df_exb.iloc[indices[0]]
    
    return related_articles


find_related_articles("I am a data scientist with experience in machine learning and data visualization")

,id,company_repName,company_name,blood_group,mobile_no,email,designation,country,password,exhibitor,role,pro_category,createdAt,updatedAt,company_profile,company_logo,is_confirmed,profile_completion,city,state,address_line1,address_line2,address_line3,fascia,website,post_code,gst,host_qty,hostess_qty,single_phase_connection,three_phase_connection,indemnity_undertaking,fever,cough,breathlessness,sore_throat_running_nose,body_ache,other_symptom,health_undertaking_accepted,exhibitor_deliverables_correct,vaccine_certificate,stall_design,stall_design_correct,alternative_email,title,first_name,last_name,oem_user_profile,oem_fascia,oem_exhibitor_badges,oem_power_requirement,oem_furniture_requirement,oem_host_hostess,oem_bare_space_stall_design,oem_booth_contractor,oem_visa,oem_indemnity_undertaking,oem_health_undertaking,oem_participation,oem_catalog_entry,company_repProfile,option_a,option_b,host,currentForm,currentFormIndex,booth_contractor_applicable,visa_applicable,firebaseToken,socket_id,catalogue_mobile_no,catalogue_email,contact_person,contact_person_designation,furniture_applicable,account_deletion_request,text,preprocessed_text,vectorized_text
124,309,Ayushi Panchal,Libratherm Instruments Private Limited,B+,+919820249855,coo@libratherm.com,COO,India,$2a$10$Dy6NkhmIBRJRoJhob8hHuez0a9wai7bn7l5WAr7...,exhibitor,exhibitor,Boilercomponentmanufacturer,2022-08-25 07:17:29,2022-09-12 10:13:31,Libratherm Instruments Private Limited is one ...,https://api.boilerworldexpo.com/images/LIPL_TM...,1,0,Mumbai,MAHARASHTRA,401-403 Diamond Industrial Estate,Ketki Pada Road,Dahisar East,LIBRATHERM INSTRUMENTS PRIVATE LIMITED,www.libratherm.com,400068,27AABCL1732A1Z5,NaN,NaN,2.0,NaN,1,0,0,0,0,0,0,1,1,NaN,NaN,0,ceo@libratherm.com,Mrs,Ayushi,Panchal,1,1,1,1,1,1,0,1,1,1,1,0,1,NaN,0,1,"""[{\""category\"":\""Basic Host\"",\""rate_per_day\...",EXHIBITOR BADGES,2,1,0,NaN,NaN,+919820249855,coo@libratherm.com,Ayushi Panchal,COO,1,0,Boilercomponentmanufacturer Libratherm Instrum...,boilercomponentmanufactur libratherm instrume...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
97,273,Ms. Akanksha Khandare,Testo India Pvt. Ltd.,A+,+917030902062,skshirsagar@testo.in,Associate Marcom,India,$2a$10$AC3Oq85D3osd4Z2J2Phvre.Wp/yzinYBqWZWWt3...,exhibitor,exhibitor,"WTP, ETP & other pollution control equipment m...",2022-08-17 05:55:49,2022-09-01 11:51:26,Testo India Pvt. Ltd. is a 100% subsidiary of ...,https://api.boilerworldexpo.com/images/Be Sure...,1,0,Pune,Maharashtra,"Plot No. 23, Sind Society,","Baner Road,","Aundh,",Testo india pvt. ltd.,www.testo.com,411007,27AACCT4593L1ZS,NaN,NaN,2.0,NaN,1,0,0,0,0,0,0,1,1,NaN,NaN,0,gjha@testo.in,Mr,Shubham,Kshirsagar,1,1,1,1,1,1,0,1,1,1,1,0,1,NaN,0,0,"""[{\""category\"":\""Basic Host\"",\""rate_per_day\...",PARTICIPATION LETTER,12,1,0,NaN,NaN,+919375342348,ppatel@testo.in,Mr. Piyush Patel,Head – Strategic Business Unit,0,0,"WTP, ETP & other pollution control equipment m...",wtp etp pollut control equip manufactur testo...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.087..."
62,186,ARUN GOYAL,GOOD LUCK INDIA LTD,O+,+91 9910496352,agoyal@goodluckindia.com,Sr. Vice President-Mktg.,India,$2a$10$TvvE.fU6/aZ.nygXjjSRpezi9bMx8vrWnch4659...,exhibitor,exhibitor,Ancillaries,2022-06-29 07:01:13,2022-09-12 08:39:32,ERW/CDW PRECISION TUBES\nMACHINED FLANGES & FO...,https://api.boilerworldexpo.com/images/logo_gl...,1,0,GAZIABAD,UTTAR PARDESH,II F 166-167,AMBEDKAR ROAD,NaN,GOOD LUCK INDIA LTD.,www.goodluckindia.com,201001,09AAACG3204D4Z5,NaN,NaN,NaN,10.0,1,0,0,0,0,0,0,1,1,NaN,NaN,0,NaN,Mr,ARUN,GOYAL,1,1,0,1,1,1,0,1,1,1,1,1,1,NaN,0,0,"""[{\""category\"":\""Basic Host\"",\""rate_per_day\...",PARTICIPATION LETTER,12,1,0,NaN,NaN,+91 9910496352,agoyal@goodluckindia.com,ARUN GOYAL,Sr. Vice President-Mktg.,0,0,Ancillaries ERW/CDW PRECISION TUBES\nMACHINED ...,ancillari erw cdw precis tube machin flang forg,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
116,300,Shubhra Parab,V T CORP PVT LTD,AB+,+919987616777,vtcsalescoordinator@vtcorp